# Projeto - Questao 01

### Avaliação da Disciplina

<b>Julio Cesar Gonçalves Sales</b>

<b>Victor Medeiros Outtes Alves</b>

<b>Disciplina:</b> Computação Evolucionária e Inteligência de Enxames

![img/q1.png](img/q1.png)

In [1]:
from sklearn import svm
from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import ShuffleSplit, cross_val_score
from deap import base
from deap import benchmarks
from deap import creator
from deap import tools
from deap import algorithms
import pylab as pl
import operator
import random
import numpy
import math
import copy
import matplotlib.pyplot as plt
random.seed(100)

### Delimitação do espaco de busca

In [2]:
cmin = 2e-15
cmax = 2e3
gmin = 2e-5
gmax = 2e13

### Leitura da base: DIGITS
A base escolhida para esta questão foi a DIGITS. É uma base de classificação de dígitos manuscritos, consistindo em 10 classes.

In [3]:
digits = datasets.load_digits()
Y = digits.target
X = digits.data

### Função de fitness
A função de fitness será um treinamento e avaliação média de um classificador SVM utilizando validação cruzada 3-fold

In [4]:
def evaluation(individuo):
    c = individuo[0]
    gamma = individuo[1]
    classificador = svm.SVC(decision_function_shape='ovo', C=c,gamma=gamma)
    validacao_cruzada = ShuffleSplit(n_splits=3)
    scores = cross_val_score(classificador, X, Y, cv=validacao_cruzada, scoring='accuracy')
    return scores.mean(),

### Plotar gráfico de convergência

In [5]:
def plot(resultados):
    fig, ax = plt.subplots()
    ax.plot(resultados[0].select('gen'),resultados[0].select('avg'),'r',label='Algoritmo Genético')
    ax.plot(resultados[1].select('gen'),resultados[1].select('avg'),'b',label='PSO')
   
    ax.set(xlabel='Geracao', ylabel='Fitness (Media)', title='Convergência')
    plt.legend(loc='lower right')
    #plt.yscale('log') #escala logaritmica
    ax.grid()
    plt.show()

### PSO

#### Função para gerar uma partícula
A partícula será formada pelos atributos:
* Vetor de características, no caso o C e o GAMMA do SVM
* C minimo e máximo
* Gamma minimo e máximo
* Velocidade

O C e GAMMA serão gerados aleatóriamente de acordo com as restrições respectivas.

In [6]:
def generate(size, cmin, cmax, gmin, gmax, s_cmin, s_cmax, s_gmin, s_gmax):
    c = random.uniform(cmin, cmax)
    gamma = random.uniform(gmin, gmax)
    part = creator.Particle([c, gamma])
    part.cmin = cmin
    part.cmax = cmax
    part.gmin = gmin
    part.gmax = gmax
    part.speed = [random.uniform(s_cmin, s_cmax),random.uniform(s_gmin, s_gmax)]
    return part

#### Função para atualizar os valores da partícula

In [7]:
def updateParticle(part, best, c1, c2): 
    c1r1 = (random.uniform(0, c1) for _ in range(len(part)))
    c2r2 = (random.uniform(0, c2) for _ in range(len(part)))
    cognitivo = map(operator.mul, c1r1, map(operator.sub, part.best, part))
    social = map(operator.mul, c2r2, map(operator.sub, best, part))

    # define a velocidade
    part.speed = map(operator.add, part.speed, map(operator.add, cognitivo, social))
    
    # gera um peso de inercia, entre 0.4 e 0.9
    inertia_weights = (random.uniform(0.4, 0.9) for _ in range(len(part)))
    
    # aplica os pesos de inercia a velocidade
    part.speed = list(map(operator.mul, inertia_weights, part.speed))
    
    # atualizacao da posicao atual pela velocidade computada
    # o operador [:] cria uma copia, para nao apontar para o ponteiro
    part[:] = list(map(operator.add, part, part.speed))
    
    # impede a particula de ser ajustada para fora do espaço de busca
    if part[0] < part.cmin:
        part[0] = part.cmin
    elif part[0] > part.cmax:
        part[0] = part.cmax
    
    if part[1] < part.gmin:
        part[1] = part.gmin
    elif part[1] > part.gmax:
        part[1] = part.gmax

#### Toolbox do PSO

In [8]:
creator.create("Fitness", base.Fitness, weights=(1.0,))
creator.create("Particle", list, fitness=creator.Fitness, 
               cmin=None, cmax=None, gmin=None, gmax=None,
               speed=list, s_cmin=None, s_cmax=None, 
               s_gmin=None, s_gmax=None, best=None)
toolboxPSO = base.Toolbox()
toolboxPSO.register("particle", generate, size=2, cmin=cmin, cmax=cmax, 
                    gmin=gmin, gmax=gmax, s_cmin=cmin*0.1, s_cmax=cmax*0.1,
                    s_gmin=gmin*0.1, s_gmax=gmax*0.1)
toolboxPSO.register("population", tools.initRepeat, list, toolboxPSO.particle)
toolboxPSO.register("update", updateParticle, c1=2.0, c2=2.0)
toolboxPSO.register("evaluate", evaluation)

In [9]:
# roda o PSO. Aqui a arquitetura de comunicacao entre as particulas serah estrela
def PSO():
    # cria uma populacao de tamanho N
    pop = toolboxPSO.population(n=20)
    
    # informa quais estatisticas quer calcular
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = ["gen", "evals"] + stats.fields
    
    # numero de iteracoes (geracoes)
    GEN = 20
    # inicializa a melhor posicao
    best = None
    
    # vetor para armazenar o melhor resultado a cada iteracao apenas para
    # efeito de plotar a convergencia
    all_bests = []

    # para cada geracao...
    for g in range(GEN):
        # para cada particula na populacao
        for part in pop:
            # avalia a particula
            part.fitness.values = toolboxPSO.evaluate(part)
            
            # atualizacao cognitiva (pessoal)
            # caso nao exista melhor posicao coginitiva anterior
            # eh a propria posicao atual
            # lembrar que o operador < significa na DEAP "PIOR QUE"
            if not part.best or part.best.fitness < part.fitness:
                part.best = creator.Particle(part)
                part.best.fitness.values = part.fitness.values
            # atualizacao social (global)
            # caso nao exista melhor posicao social anterior
            # eh a propria posicao cognitiva atual
            if not best or best.fitness < part.fitness:
                best = creator.Particle(part)
                best.fitness.values = part.fitness.values
        
        #fim da avaliacao da populacao
        
        all_bests.append(best.fitness.values)
        
        # atualiza cada particula
        for part in pop:
            toolboxPSO.update(part, best)

        # Gather all the fitnesses in one list and print the stats
        logbook.record(gen=g, evals=len(pop), **stats.compile(pop))
        print(logbook.stream)
        #print('Geracao: ' + str(g) + ' - Melhor Fitness: ' + str(best.fitness.values) + ' (' + str(best) + ')')
    
    return pop, logbook, best, all_bests

### Algoritmo Genético

#### Crossover e Mutação adaptados ao problema

In [10]:
def crossover(ind1,ind2):
    temp = copy.deepcopy(ind1)
    ind1[0] = ind2[0]
    ind2[0] = temp[0]
    return ind1,ind2,

In [11]:
def mutacao(individual, indpb):
    mu_c = numpy.mean([cmin,cmax])
    mu_gamma = numpy.mean([gmin,gmax])
    sigma_c = numpy.std([cmin,cmax])
    sigma_gamma = numpy.std([gmin,gmax])
    #MU
    if random.random() > indpb:
        individual[0] += random.gauss(mu_c, sigma_c)
        if individual[0] < cmin:
            individual[0] = cmin
        elif individual[0] > cmax:
            individual[0] = cmax
            
    #GAMMA
    if random.random() > indpb:
        individual[1] += random.gauss(sigma_c, sigma_gamma)
        if individual[1] < gmin:
            individual[1] = gmin
        elif individual[1] > gmax:
            individual[1] = gmax
    return individual,

#### Toolbox do GA

In [12]:
creator.create("FitnessGA", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessGA)
toolboxGA = base.Toolbox()
toolboxGA.register("attr_c", random.uniform, cmin, cmax)
toolboxGA.register("attr_gamma", random.uniform, gmin, gmax)
toolboxGA.register("individual", tools.initCycle, creator.Individual, (toolboxGA.attr_c, toolboxGA.attr_gamma), 1)
toolboxGA.register("population", tools.initRepeat, list, toolboxGA.individual)
toolboxGA.register("evaluate", evaluation)
toolboxGA.register("mate", crossover)
toolboxGA.register("mutate", mutacao, indpb=0.25)
toolboxGA.register("select", tools.selTournament, tournsize=3)

#### Definições do GA

In [13]:
def GA():
    pop = toolboxGA.population(n=20)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    pop, log = algorithms.eaSimple(pop, toolboxGA, cxpb=0.5, mutpb=0.5, ngen=20, 
                                   stats=stats, halloffame=hof, verbose=True)
    return hof[0],log

### Execução do código

In [14]:
if __name__ == "__main__":
    resultsGA = GA()
    print('Melhor resultado GA: ',resultsGA[0])
    print('Fitness do melhor resultado GA: ',evaluation(resultsGA[0]))
    print('==================================================================================================')
    resultsPSO = PSO()
    print('Melhor resultado PSO: ',resultsPSO[2])
    print('Fitness do melhor resultado PSO: ',evaluation(resultsPSO[2]))
    
    plot([resultsGA[1], resultsPSO[1]])

gen	nevals	avg      	std       	min      	max      
0  	20    	0.0716667	0.00750171	0.0537037	0.0833333
1  	16    	0.0736111	0.00697524	0.0574074	0.0833333
2  	16    	0.0725926	0.00684434	0.0648148	0.0888889
3  	14    	0.118519 	0.199779  	0.0611111	0.988889 
4  	17    	0.207593 	0.326961  	0.0611111	0.990741 
5  	11    	0.345648 	0.415482  	0.0592593	0.987037 
6  	14    	0.708889 	0.41443   	0.0592593	0.988889 
7  	15    	0.938148 	0.197027  	0.0796296	0.990741 
8  	18    	0.574352 	0.453663  	0.062963 	0.988889 
9  	15    	0.802222 	0.365265  	0.062963 	0.988889 
10 	18    	0.847222 	0.325611  	0.062963 	0.990741 
11 	17    	0.756481 	0.394234  	0.0666667	0.992593 
12 	16    	0.709352 	0.416928  	0.0685185	0.992593 
13 	14    	0.753056 	0.395915  	0.0537037	0.992593 
14 	15    	0.710556 	0.416299  	0.0703704	0.992593 
15 	11    	0.84713  	0.325297  	0.062963 	0.990741 
16 	13    	0.846852 	0.322859  	0.0703704	0.990741 
17 	17    	0.801667 	0.367326  	0.0611111	0.992593 
18 	15    	0